This code produces a polygon from the DWD 1x1km grid.

In [50]:
from netCDF4 import Dataset, MFDataset
import numpy as np
from datetime import datetime, timedelta
from cftime import num2date, date2num, date2index
import geopandas as gpd
from shapely.geometry import Polygon
from climate_data.lat_lon_mask import lat_lon_mask as mask
import json

In [77]:
nc = Dataset('climate_netcdf_forecast/combined.nc', 'r')

In [52]:
nc.variables.keys()

dict_keys(['time', 'time_bnds', 'lon', 'lat', 'height', 'hurs', 'pr', 'rsds', 'sfcWind', 'tas', 'tasmax', 'tasmin'])

In [78]:
forecast_lats = nc.variables['lat'][:]
forecast_lons = nc.variables['lon'][:]

In [79]:
mask = np.ma.getmask(nc['hurs'][0])

# # Convert the mask to a boolean array (True for masked/missing data, False otherwise)
boolean_mask = np.logical_not(mask.astype(bool))

In [80]:
boolean_mask[50][50]

True

In [81]:
forecast_mask = {
    'lat': forecast_lats.tolist(),
    'lon': forecast_lons.tolist(),
    'mask': boolean_mask.tolist()
}

In [82]:
with open('climate_netcdf_forecast/forecast_mask.py', 'w') as f:
    json.dump(forecast_mask, f)


In [83]:
# Geojson
# create point_geojson
point_geojson = {
    "type": "FeatureCollection",
    "features": []
}

for i in range(len(forecast_lats)):
    for j in range(len(forecast_lons)):
        if boolean_mask[i, j]:
            point_geojson['features'].append({
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [forecast_lons[j], forecast_lats[i]]
                },
                "properties": {
                    "lon": forecast_lons[j],
                    "lat": forecast_lats[i],
                    "lon_idx": j,
                    "lat_idx": i
                }
            })

In [36]:
with open('climate_netcdf_forecast/forecast_points.geojson', 'w') as f:
    json.dump(point_geojson, f)

In [84]:
# create polygon_geojson
lat_diffs = np.diff(forecast_lats)
lon_diffs = np.diff(forecast_lons)
lon_diff = lon_diffs.mean()
lat_diff = lat_diffs.mean()

In [90]:
polygon_geojson = {
    "type": "FeatureCollection",
    "features": []
}

for p in point_geojson["features"]:
    polygon_geojson["features"].append({
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [[
                [p["geometry"]["coordinates"][0] - lon_diff/2, p["geometry"]["coordinates"][1] - lat_diff/2],
                [p["geometry"]["coordinates"][0] + lon_diff/2, p["geometry"]["coordinates"][1] - lat_diff/2],
                [p["geometry"]["coordinates"][0] + lon_diff/2, p["geometry"]["coordinates"][1] + lat_diff/2],
                [p["geometry"]["coordinates"][0] - lon_diff/2, p["geometry"]["coordinates"][1] + lat_diff/2],
                [p["geometry"]["coordinates"][0] - lon_diff/2, p["geometry"]["coordinates"][1] - lat_diff/2],
            ]]
        },
        "properties": p["properties"]
    })

In [92]:
with open('climate_netcdf_forecast/forecast_polygons.geojson', 'w') as f:
    json.dump(polygon_geojson, f)